In [45]:
def filter_responses(responses, requested_fields):
    def extract_fields(data, fields):

        if isinstance(data, dict):
            result = {}
            for field in fields:
                keys = field.split(".")
                value = data
                for key in keys:
                    if isinstance(value, dict) and key in value:
                        value = value[key]
                    else:
                        value = None
                        break
                if value is not None:
                    result[keys[-1]] = value
            return result
        return {}

    # Apply extraction to each dictionary in the source list
    filtered_responses_list = [extract_fields(item, requested_fields) for item in responses]
    return filtered_responses_list

In [46]:
import os
import csv
import json

from FirewallAPI import Firewall, EQ, NOT, LIKE

firewall_ip = "172.16.17.30"
username = "apiadmin"
password = "883C4BBDEF0B32DA7176B651133A52C1F215E15696EFD7D3506EB924A0EC0097"
# password="Sophos1985"
password_encrypted = False

firewall = Firewall(username, password=password, hostname=firewall_ip, port=4444, certificate_verify=False, password_encrypted=True)

entity_type = "FirewallRule"
responses = firewall.read(entity_type)

In [47]:
responses

{'status': '216',
 'message': 'Operation Successful.',
 'data': [{'Name': 'IPsec Sophos_To_FortiGate',
   'Description': 'Auto created Firewall Rule',
   'IPFamily': 'IPv4',
   'Status': 'Enable',
   'Position': 'Top',
   'PolicyType': 'Network',
   'NetworkPolicy': {'Action': 'Accept',
    'LogTraffic': 'Enable',
    'SkipLocalDestined': 'Disable',
    'SourceZones': {'Zone': ['VPN', 'LAN']},
    'DestinationZones': {'Zone': ['VPN', 'LAN']},
    'Schedule': 'All The Time',
    'SourceNetworks': {'Network': ['192.168.30.0/24', '192.168.10.0/24']},
    'DestinationNetworks': {'Network': ['192.168.30.0/24', '192.168.10.0/24']},
    'DSCPMarking': '-1',
    'WebFilter': 'None',
    'WebCategoryBaseQoSPolicy': None,
    'BlockQuickQuic': 'Disable',
    'ScanVirus': 'Disable',
    'ZeroDayProtection': 'Disable',
    'ProxyMode': 'Disable',
    'DecryptHTTPS': 'Disable',
    'ApplicationControl': 'None',
    'ApplicationBaseQoSPolicy': None,
    'IntrusionPrevention': 'None',
    'TrafficSha

In [51]:
for item in responses['data']:
    if item['PolicyType']=="User":
        print(json.dumps(item,indent=4))

{
    "Name": "Traffic to Internal Zones",
    "Description": "A disabled Firewall rule with the destination zone as LAN, WiFi, VPN or DMZ . Such rules would be added to Traffic to Internal Zones group on the first match basis if user selects automatic grouping option.",
    "IPFamily": "IPv4",
    "Status": "Enable",
    "Position": "After",
    "PolicyType": "User",
    "After": {
        "Name": "DNAT to srv.lab.local"
    },
    "UserPolicy": {
        "Action": "Drop",
        "LogTraffic": "Enable",
        "DestinationZones": {
            "Zone": [
                "DMZ",
                "VPN",
                "WiFi",
                "LAN"
            ]
        },
        "Schedule": "All The Time",
        "SkipLocalDestined": "Disable",
        "MatchIdentity": "Enable",
        "DataAccounting": "Disable",
        "ShowCaptivePortal": "Enable"
    }
}
{
    "Name": "Clone_Default WAN Access",
    "Description": null,
    "IPFamily": "IPv4",
    "Status": "Enable",
    "Positi

In [ ]:
requested_fields = [
    "Customer",
    "Name",
    "PolicyType",
    "NetworkPolicy.Action",
    "NetworkPolicy.SourceZones",
    "NetworkPolicy.DestinationZones",
    "NetworkPolicy.SourceNetworks",
    "NetworkPolicy.DestinationNetworks",
    "NetworkPolicy.Services",
    "UserPolicy.Action",
    "UserPolicy.SourceZones",
    "UserPolicy.DestinationZones",
    "UserPolicy.SourceNetworks",
    "UserPolicy.DestinationNetworks",
    "UserPolicy.Services",
    "UserPolicy.Identity",
    "HTTPBasedPolicy.HostedAddress",
    "HTTPBasedPolicy.HTTPS",
    "HTTPBasedPolicy.ListenPort",
    "HTTPBasedPolicy.Domains",
    "HTTPBasedPolicy.AccessPaths",
]


filtered_data = filter_responses(responses["data"], requested_fields)

In [ ]:
filtered_data

In [ ]:
for item in filtered_data:
    if item['PolicyType']=="User":
        print(item)

In [ ]:
customer_field = "Customer"
customer_name = "Integrity 360 Test"

# Using list comprehension to add the key-value pair
filtered_data = [{**item, customer_field: customer_name} for item in filtered_data]

In [ ]:
# Requested fields
write_fields = [
    "Customer",
    "Name",
    "PolicyType",
    "Action",
    "SourceZones",
    "DestinationZones",
    "Identity",
    "SourceNetworks",
    "DestinationNetworks",
    "Services",
    "Identity",
    "HostedAddress",
    "HTTPS",
    "ListenPort",
    "Domains",
    "AccessPaths",
]

with open(f"Test_{entity_type.upper()}.csv", "w", encoding="UTF8", newline="") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=write_fields, extrasaction="ignore", restval="...")
    writer.writeheader()
    writer.writerows(filtered_data)